In [50]:
import json
%reload_ext e2emlstorlets.tools.ipython

In [42]:
import os
os.environ['OS_AUTH_VERSION'] = '3'
os.environ['OS_AUTH_URL'] = 'http://127.0.0.1:5000/v3'
os.environ['OS_USERNAME'] = 'tester'
os.environ['OS_PASSWORD'] = 'testing'
os.environ['OS_USER_DOMAIN_NAME'] = 'default'
os.environ['OS_PROJECT_DOMAIN_NAME'] = 'default'
os.environ['OS_PROJECT_NAME'] = 'test'

In [37]:
%%storletapp extract_face.ExtractFace
import cv2
import numpy as np

def detect(im):
    mat=cv2.imdecode(im, cv2.IMREAD_GRAYSCALE)
    cascade = cv2.CascadeClassifier("/usr/local/share/OpenCV/haarcascades/haarcascade_frontalface_alt.xml")
    rects = cascade.detectMultiScale(mat)

    if len(rects) == 0:
        return [], mat
    rects[:, 2:] += rects[:, :2]
    rect = rects[0]
    return mat, rect

def crop(img, rect):
    h = rect[3]-rect[1]
    w = rect[2]-rect[0]
    x = rect[0]
    y = rect[1]
    return img[y:y+h, x:x+w]

class ExtractFace(object):
    def __init__(self, logger):
        self.logger = logger

    def __call__(self, in_files, out_files, params):
        metadata = in_files[0].get_metadata()
        metadata['name'] = params.get('name', '')
        out_files[0].set_metadata(metadata)

        # Read the image
        img_str = ''
        while True:
            buf = in_files[0].read(1024)
            if not buf:
                break
            img_str += buf
        img_nparr = np.fromstring(img_str, np.uint8)
        
        # Detect face
        mat, rect = detect(img_nparr)

        # Crop the face and decrease resolution
        face = crop(mat, rect)
        small_face = cv2.resize(face, (30,30))
        
        # Write result
        retval, small_face_buf = cv2.imencode('.jpg', small_face)
        out_files[0].write(small_face_buf)
        
        in_files[0].close()
        out_files[0].close()
        self.logger.debug('Done\n')

Upload storlets succeeded /storlet/extract_face.py
Example command `swift download <container> <object> -H X-Run-Storlet:extract_face.py`


In [38]:
# Iterate over all puctures, and extract faces
%list_container -i tr -o obj_list
for obj in obj_list:
    input_path=os.path.join('/tr',obj)
    output_path=os.path.join('/str',obj)
    %copy --storlet extract_face.py --input path:$input_path --output path:$output_path -o result

In [52]:
# Train the model with all faces and name to id dictionary
name_to_id = {'bibi': 1, 'merkel': 2, 'obama': 3, 'trump': 4}
params = {'name_to_id': json.dumps(name_to_id)}
input_path = os.path.join('path:str/', obj_list[0])
extra_resources = ','.join([ '/str/%s' % obj_name for obj_name in obj_list[1:]])
output_path = 'path:/trained/model'
%copy --storlet train_model.py --input $input_path --output $output_path --extra $extra_resources -i params -o result

ClientException: Object COPY failed: http://127.0.0.1:8080/v1/AUTH_a87b32da2ddf40cda6e1d7fc78da181e/str/bibi1.jpeg 500 Internal Error   Storlet execution failed

In [45]:
obj_list[0]

u'bibi1.jpeg'